In [ ]:
import nltk
# nltk.download('punkt', download_dir='NLTK_DATA')
# nltk.download('stopwords', download_dir='NLTK_DATA')
nltk.data.path.append("NLTK_DATA")
import pandas as pd
import numpy as np
from processing import freqs, normalisasi
from sentiment import *

# PREPARASI *TRAINING DATASET*

In [ ]:
# TODO: pindah ke sqlite
dataset = pd.read_csv("dataset/training_dataset.csv")
all_pos = dataset[dataset['label'] == 1]
all_neg = dataset[dataset['label'] == 0]

# * Training data: 80% pertama
# * Testing data: 20% terakhir
train_pos = all_pos[:int(len(all_pos)*0.8)]
train_neg = all_neg[:int(len(all_neg)*0.8)]
test_pos = all_pos[-int(len(all_pos)*0.2):]
test_neg = all_neg[-int(len(all_neg)*0.2):]

train_x = train_pos.append(train_neg, ignore_index=True)
test_x = test_pos.append(test_neg, ignore_index=True)

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [ ]:
ld = len(dataset)
trp = len(train_pos)
tng = len(train_neg)
tsp = len(test_pos)
tsn = len(test_neg)
print(f"dataset: {ld} \n train pos: {trp} \n train neg: {tng} \n test pos: {tsp} \n test neg: {tsn}")

In [ ]:
len(all_pos)

# PELATIHAN *MODEL CLASSIFIER*

In [ ]:
x = train_x['text'].tolist()
y = train_y.tolist()
kamus_frek = kamus_freq(x, y)

In [ ]:
for x in kamus_frek.keys():
    print(kamus_frek.get(x, 0))

## logprior
$D$ = jumlah dokumen <br>
$D_{pos}$ = jumlah dokumen positif <br>
$D_{neg}$ = jumlah dokumen negatif <br>
### probabilitas dokumen positif
$P(D_{pos}) = \frac{D_{pos}}{D}$
### probabilitas dokumen negatif
$P(D_{neg}) = \frac{D_{neg}}{D}$
### probabilitas prior
${logprior} = log \left( \frac{P(D_{pos})}{P(D_{neg})} \right) = log \left( \frac{D_{pos}}{D_{neg}} \right)$
<br> disederhanakan, menjadi: <br>
$logprior = P(D_{pos}) - P(D_{neg})$

## loglikelihood
$W$ = probabilitas positif/negatif kata <br>
$freq$ = frekuensi kata unik yang bernilai positif/negatif <br>
$N$ =  jumlah total kata positif/negatif <br>
$V$ = jumlah total kata unik (panjang dokumen frekuensi) <br>
### probabilitas kata positif
$P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}$
### probabilitas kata negatif
$P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V}$
### likelihood kata
$\text{loglikelihood} = \log \left(\frac{P(W_{pos})}{P(W_{neg})} \right)$

In [ ]:
# logprior, loglikelihood = train_nbc(kamus_frek, train_y)
logprior = hitungPrior(kamus_frek, train_y)
loglikelihood = hitungLikelihood(kamus_frek)
# print(logprior)
# print(loglikelihood)

## HASIL PENGUJIAN MODEL
- data *training*
- data *testing*

In [12]:
test = test_nbc(test_x['text'].tolist(), test_y.tolist(), logprior, loglikelihood)
test

0.8455598455598455

## UJI KESELURUHAN *DATASET*

In [ ]:
data_test = test_nbc(dataset['text'], dataset['label'], logprior, loglikelihood)
data_test

## *CONFUSION MATRIX*

## metrik-metrik
-   TP = True Positive
-   TN = True Negative
-   FP = False Positive
-   FN = False Negative
- Populasi = TP+TN+FP+FN
<hr>

### akurasi

jumlah prediksi yang benar. tidak dianjurkan sebagai metrik kunci. <br>
$\frac{TP+TN}{Populasi}$

### miss

jumlah prediksi yang salah <br>
$\frac{FP+FN}{Populasi}$

### presisi

persentase positif yang diprediksi secara akurat. jika FP = 0, maka nilai presisi 100%. <br>
$\frac{TP}{TP+FP}$

### sensitifitas

nilai positif sesungguhnya <br>
$\frac{TP}{TP+FN}$

In [ ]:
pdc = pd.DataFrame()
pdc = dataset
pdc['label_prediksi'] = pdc['text'].apply(lambda x: predict_nbc(x, logprior, loglikelihood))

In [ ]:
# TODO: Do more testing with thresholds values from 0, 0.5, 0.8, 0.9, 1
# * to note;
# treshold = -0.03272669350255697
# * default treshold is 0
def toLabel(x):
   if x > 0:
       return 1
   elif x < 0:
       return 0
   else:
       return "n"
pdc['prediksi'] = pdc['label_prediksi'].apply(lambda x: toLabel(x))

In [ ]:
pdc

In [ ]:
tr = logprior
def toLabel(x):
   if x > tr:
       return 1
   elif x < tr:
       return 0
   else:
       return "n"
pdc['prediksi_tr'] = pdc['label_prediksi'].apply(lambda x: toLabel(x))

In [ ]:
conf_matrix = pd.crosstab(pdc['label'], pdc['prediksi'], rownames=['Actual'], colnames=['Predicted'])
conf_matrix

In [ ]:
pop = 647+555+14+84
a = (647+555)/pop
m = (14+84)/pop
p = 555/(555+14)
s = 555/(555+84)
print(f"pop={pop}\nakurasi:{a}, miss:{m}, presisi: {p}, sensi: {s}.\n F1:{2*(p*s)/(p+s)}")

In [ ]:
pdc

In [ ]:
acc = 0
for index, row in pdc.iterrows():
    if row['prediksi'] == row['label']:
        acc+=1

acc

In [ ]:
conf2 = pd.crosstab(pdc['label'], pdc['prediksi_tr'], rownames=['Actual'], colnames=['Predicted'])
conf2

In [ ]:
a2 = (647+556)/1300
m2 = (14+83)/1300
p2 = 556/(556+14)
s2 = 556/(556+83)
print(f"akurasi:{a2}, miss:{m2}, presisi: {p2}, sensi: {s2}.\n F1:{2*(p2*s2)/(p2+s2)}")

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

sn.heatmap(conf_matrix, annot=True, fmt='g')
plt.show()

In [ ]:
sn.heatmap(conf2, annot=True, fmt='g')
plt.show()

# UJI CLASSIFIER PADA ARTIKEL BERITA

In [ ]:
berita = open('berita.txt', 'r')
berita = berita.read()

In [ ]:
beritanormal = normalisasi(berita)
frekberita = freqs(beritanormal)
# frekberita

## SENTIMEN ARTIKEL KESELURUHAN

In [ ]:
predict_nbc(berita, logprior, loglikelihood)

## SENTIMEN ARTIKEL PER_KALIMAT

In [ ]:
from nltk.tokenize import sent_tokenize
x = pd.DataFrame()
kalimat = sent_tokenize(berita)
x['kalimat'] = kalimat
x['kalimat_normal'] = x['kalimat'].apply(lambda x: normalisasi(x))

In [ ]:
x['label_prediksi'] = x['kalimat'].apply(lambda x: predict_nbc(x, logprior, loglikelihood))

In [ ]:
x['prediksi'] = x['label_prediksi'].apply(lambda x: toLabel(x))
x

## SENTIMEN ARTIKEL PER_KATA

In [ ]:
berita_normal = normalisasi(berita)

In [ ]:
# * predict_nbc per-kata
def predict_perkata(text, logpri, loglik):
    l = []
    for w in text:
        x = predict_nbc(w, logpri, loglik)
        l.append({
            "kata":w,
            "skor":x
            })
    return l

In [ ]:
hasil = predict_perkata(berita_normal, logprior, loglikelihood)
type(hasil[0])

In [ ]:
hasil

In [ ]:
logprior

In [ ]:
import csv
keys = hasil[0].keys()

with open('demo-kata.csv', 'w', newline='') as file:
    dict_writer = csv.DictWriter(file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(hasil)